In [4]:
import numpy as np

def pearsonr_np1(x, y, nan_policy="omit"):
    """
    Compute Pearson correlation coefficient r between x and y.

    Parameters
    ----------
    x, y : array-like
        1D arrays (or anything array-like) of equal length.
    nan_policy : {"omit", "raise", "propagate"}
        - "omit": drop pairs where x or y is NaN (default).
        - "raise": raise a ValueError if any NaN is present.
        - "propagate": return NaN if any NaN is present.

    Returns
    -------
    r : float
        Pearson correlation coefficient in [-1, 1], or NaN if undefined.
    n : int
        Number of paired samples actually used in the calculation.
    """
    x = np.asarray(x, dtype=float).ravel()
    y = np.asarray(y, dtype=float).ravel()
    if x.size != y.size:
        raise ValueError(f"Length mismatch: len(x)={x.size}, len(y)={y.size}")

    # NaN handling
    if nan_policy not in {"omit", "raise", "propagate"}:
        raise ValueError("nan_policy must be 'omit', 'raise', or 'propagate'")
    has_nan = np.isnan(x) | np.isnan(y)
    if has_nan.any():
        if nan_policy == "raise":
            raise ValueError("NaNs found in inputs.")
        if nan_policy == "propagate":
            return np.nan, 0
        # omit
        keep = ~has_nan
        x, y = x[keep], y[keep]

    n = x.size
    if n < 2:
        return np.nan, n

    # Center
    x = x - x.mean()
    y = y - y.mean()

    # Std devs (L2 norms of centered vectors)
    sx = np.sqrt(np.dot(x, x))
    sy = np.sqrt(np.dot(y, y))
    if sx == 0.0 or sy == 0.0:
        # one of the variables is constant
        return np.nan, n

    r = float(np.dot(x, y) / (sx * sy))
    # clamp tiny numerical overshoot
    if r > 1: r = 1.0
    if r < -1: r = -1.0
    return r, n
    


In [ ]:
x3 = [5.000000000000000278e-02,
      0.10,
      1.500000000000000222e-01,
      2.000000000000000111e-01, 
      2.500000000000000000e-01,
      2.999999999999999889e-01,
      3.499999999999999778e-01,
      3.999999999999999667e-01, 
      4.499999999999999556e-01, 
      4.999999999999999445e-01]

y1 = [1.500000000000000000e+00, 
      4.000000000000000000e+00, 
      5.333333333333333037e+00, 
      4.333333333333333037e+00, 
      1.846153846153846256e+00, 
      7.250000000000000000e+00, 
      6.333333333333333037e+00, 
      2.166666666666666519e+00, 
      1.000000000000000000e+00, 
      2.500000000000000000e+00]

r1, n1 = pearsonr_np1(x3, y1)
r1, n1

In [23]:
import pandas as pd
import numpy as np


# robust Pearson (same as I gave earlier)
def pearsonr_np(x, y):
    x = np.asarray(x, float).ravel()
    y = np.asarray(y, float).ravel()
    # drop NaN pairs
    m = ~(np.isnan(x) | np.isnan(y))
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, x.size
    x = x - x.mean(); y = y - y.mean()
    sx = np.sqrt(np.dot(x, x)); sy = np.sqrt(np.dot(y, y))
    if sx == 0.0 or sy == 0.0:
        return np.nan, x.size
    r = float(np.dot(x, y) / (sx * sy))
    return max(-1.0, min(1.0, r)), x.size

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["avg_time"].values)
    return pd.Series({"r": r, "n": n})




In [24]:
# --- your CSV ---
df = pd.read_csv("./experimental_data/R0_sigma_Dimmunity_results_v2.csv")   # columns: R0, sigma, Dimmunity

In [19]:
res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_avg_time.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)

       R0  sigma         r     n
0     1.0    0.1 -0.139555  10.0
1     1.0    0.2 -0.376905  10.0
2     1.0    0.3  0.365915  10.0
3     1.0    0.4  0.449635  10.0
4     1.0    0.5 -0.073054  10.0
..    ...    ...       ...   ...
115  12.0    0.6 -0.454116  10.0
116  12.0    0.7  0.322152  10.0
117  12.0    0.8 -0.189617  10.0
118  12.0    0.9 -0.031257  10.0
119  12.0    1.0  0.259944  10.0

[120 rows x 4 columns]
(12, 10)


In [20]:
# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["avg_time"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_avg_time.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)

    sigma  Dimmunity         r     n
0     0.1       0.05  0.664043  12.0
1     0.1       0.10  0.704008  12.0
2     0.1       0.15  0.835288  12.0
3     0.1       0.20  0.581389  12.0
4     0.1       0.25  0.692111  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.621392  12.0
96    1.0       0.35  0.716241  12.0
97    1.0       0.40  0.656650  12.0
98    1.0       0.45  0.719450  12.0
99    1.0       0.50  0.717877  12.0

[100 rows x 4 columns]
(10, 10)


In [21]:
# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["avg_time"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_avg_time.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)

       R0  Dimmunity         r     n
0     1.0       0.05  0.707897  10.0
1     1.0       0.10 -0.029625  10.0
2     1.0       0.15 -0.150877  10.0
3     1.0       0.20 -0.433921  10.0
4     1.0       0.25  0.259899  10.0
..    ...        ...       ...   ...
115  12.0       0.30  0.595496  10.0
116  12.0       0.35  0.307714  10.0
117  12.0       0.40  0.392464  10.0
118  12.0       0.45  0.645583  10.0
119  12.0       0.50  0.684382  10.0

[120 rows x 4 columns]
(12, 10)


In [22]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["max_time"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_max_time.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["max_time"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_max_time.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["max_time"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_max_time.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.480384  12.0
1     0.1       0.10  0.480384  12.0
2     0.1       0.15  0.480384  12.0
3     0.1       0.20  0.480384  12.0
4     0.1       0.25  0.480384  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.480384  12.0
96    1.0       0.35  0.502783  12.0
97    1.0       0.40  0.546582  12.0
98    1.0       0.45  0.587534  12.0
99    1.0       0.50  0.501693  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.742658  10.0
1     1.0       0.10 -0.413047  10.0
2     1.0       0.15 -0.374903  10.0
3     1.0       0.20 -0.599524  10.0
4     1.0       0.25  0.065084  10.0
..    ...        ...       ...   ...
115  12.0       0.30       NaN  10.0
116  12.0       0.35       NaN  10.0
117  12.0       0.40       NaN  10.0
118  12.0       0.45       NaN  10.0
119  12.0       0.50       NaN  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [25]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["num_strains"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_num_strains.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["num_strains"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_num_strains.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["num_strains"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_num_strains.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.939391  12.0
1     0.1       0.10  0.850174  12.0
2     0.1       0.15  0.765790  12.0
3     0.1       0.20  0.976942  12.0
4     0.1       0.25  0.887665  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.681372  12.0
96    1.0       0.35  0.602068  12.0
97    1.0       0.40  0.713581  12.0
98    1.0       0.45  0.708755  12.0
99    1.0       0.50  0.657449  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05 -0.454857  10.0
1     1.0       0.10  0.289749  10.0
2     1.0       0.15  0.309404  10.0
3     1.0       0.20 -0.180618  10.0
4     1.0       0.25 -0.544235  10.0
..    ...        ...       ...   ...
115  12.0       0.30  0.406181  10.0
116  12.0       0.35  0.839782  10.0
117  12.0       0.40  0.625286  10.0
118  12.0       0.45  0.570914  10.0
119  12.0       0.50  0.522233  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [26]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["avg_time_repeat"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_avg_time_repeat.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["avg_time_repeat"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_avg_time_repeat.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["avg_time_repeat"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_avg_time_repeat.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05 -0.503869  12.0
1     0.1       0.10 -0.531673  12.0
2     0.1       0.15 -0.512587  12.0
3     0.1       0.20 -0.568280  12.0
4     0.1       0.25 -0.526513  12.0
..    ...        ...       ...   ...
95    1.0       0.30 -0.531640  12.0
96    1.0       0.35 -0.648564  12.0
97    1.0       0.40 -0.532957  12.0
98    1.0       0.45 -0.520889  12.0
99    1.0       0.50 -0.528721  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05 -0.294672  10.0
1     1.0       0.10  0.061595  10.0
2     1.0       0.15 -0.346184  10.0
3     1.0       0.20 -0.137705  10.0
4     1.0       0.25 -0.104503  10.0
..    ...        ...       ...   ...
115  12.0       0.30  0.013479  10.0
116  12.0       0.35  0.449231  10.0
117  12.0       0.40  0.475404  10.0
118  12.0       0.45 -0.320729  10.0
119  12.0       0.50  0.080470  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [27]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["var_time_repeat"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_var_time_repeat.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["var_time_repeat"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_var_time_repeat.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["var_time_repeat"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_var_time_repeat.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05 -0.613811  12.0
1     0.1       0.10 -0.551940  12.0
2     0.1       0.15 -0.446205  12.0
3     0.1       0.20 -0.457584  12.0
4     0.1       0.25 -0.589839  12.0
..    ...        ...       ...   ...
95    1.0       0.30 -0.512921  12.0
96    1.0       0.35 -0.683260  12.0
97    1.0       0.40 -0.486819  12.0
98    1.0       0.45 -0.533870  12.0
99    1.0       0.50 -0.520323  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.006684  10.0
1     1.0       0.10  0.093788  10.0
2     1.0       0.15 -0.398235  10.0
3     1.0       0.20 -0.124297  10.0
4     1.0       0.25  0.032746  10.0
..    ...        ...       ...   ...
115  12.0       0.30 -0.137314  10.0
116  12.0       0.35  0.527463  10.0
117  12.0       0.40  0.445163  10.0
118  12.0       0.45 -0.477055  10.0
119  12.0       0.50 -0.323158  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [28]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["avg_prev"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_avg_prev.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["avg_prev"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_avg_prev.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["avg_prev"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_avg_prev.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.941145  12.0
1     0.1       0.10  0.944372  12.0
2     0.1       0.15  0.944406  12.0
3     0.1       0.20  0.948332  12.0
4     0.1       0.25  0.943947  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.957408  12.0
96    1.0       0.35  0.965112  12.0
97    1.0       0.40  0.968285  12.0
98    1.0       0.45  0.966931  12.0
99    1.0       0.50  0.966437  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.585674  10.0
1     1.0       0.10 -0.181111  10.0
2     1.0       0.15 -0.003672  10.0
3     1.0       0.20 -0.659178  10.0
4     1.0       0.25  0.086982  10.0
..    ...        ...       ...   ...
115  12.0       0.30 -0.969019  10.0
116  12.0       0.35 -0.991497  10.0
117  12.0       0.40 -0.987026  10.0
118  12.0       0.45 -0.976358  10.0
119  12.0       0.50 -0.980222  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [29]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["var_prev"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_var_prev.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["var_prev"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_var_prev.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["var_prev"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_var_prev.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.995776  12.0
1     0.1       0.10  0.994496  12.0
2     0.1       0.15  0.994962  12.0
3     0.1       0.20  0.996626  12.0
4     0.1       0.25  0.994549  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.996882  12.0
96    1.0       0.35  0.997052  12.0
97    1.0       0.40  0.997389  12.0
98    1.0       0.45  0.996684  12.0
99    1.0       0.50  0.997204  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.565932  10.0
1     1.0       0.10 -0.323062  10.0
2     1.0       0.15  0.266213  10.0
3     1.0       0.20 -0.688551  10.0
4     1.0       0.25  0.047177  10.0
..    ...        ...       ...   ...
115  12.0       0.30 -0.929421  10.0
116  12.0       0.35 -0.970975  10.0
117  12.0       0.40 -0.977343  10.0
118  12.0       0.45 -0.965266  10.0
119  12.0       0.50 -0.986765  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [30]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["avg_div"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_avg_div.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["avg_div"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_avg_div.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["avg_div"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_avg_div.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.912951  12.0
1     0.1       0.10  0.898347  12.0
2     0.1       0.15  0.909725  12.0
3     0.1       0.20  0.942605  12.0
4     0.1       0.25  0.895240  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.707826  12.0
96    1.0       0.35  0.737297  12.0
97    1.0       0.40  0.741142  12.0
98    1.0       0.45  0.779929  12.0
99    1.0       0.50  0.762686  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.522706  10.0
1     1.0       0.10  0.127231  10.0
2     1.0       0.15  0.036095  10.0
3     1.0       0.20 -0.624366  10.0
4     1.0       0.25 -0.007985  10.0
..    ...        ...       ...   ...
115  12.0       0.30  0.748789  10.0
116  12.0       0.35  0.905753  10.0
117  12.0       0.40  0.692882  10.0
118  12.0       0.45  0.655587  10.0
119  12.0       0.50  0.611862  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [31]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["var_div"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_var_div.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["var_div"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_var_div.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["var_div"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_var_div.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.324777  12.0
1     0.1       0.10 -0.065202  12.0
2     0.1       0.15 -0.048465  12.0
3     0.1       0.20  0.017488  12.0
4     0.1       0.25 -0.160741  12.0
..    ...        ...       ...   ...
95    1.0       0.30 -0.618240  12.0
96    1.0       0.35 -0.607372  12.0
97    1.0       0.40 -0.526631  12.0
98    1.0       0.45 -0.548650  12.0
99    1.0       0.50 -0.457919  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.477533  10.0
1     1.0       0.10 -0.203302  10.0
2     1.0       0.15  0.514662  10.0
3     1.0       0.20 -0.586834  10.0
4     1.0       0.25 -0.213815  10.0
..    ...        ...       ...   ...
115  12.0       0.30  0.294814  10.0
116  12.0       0.35  0.565272  10.0
117  12.0       0.40  0.172572  10.0
118  12.0       0.45  0.379459  10.0
119  12.0       0.50  0.284683  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [32]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["max_abundance"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_max_abundance.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["max_abundance"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_max_abundance.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["max_abundance"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_max_abundance.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.645311  12.0
1     0.1       0.10  0.430726  12.0
2     0.1       0.15  0.604760  12.0
3     0.1       0.20  0.416311  12.0
4     0.1       0.25  0.644560  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.836041  12.0
96    1.0       0.35  0.906734  12.0
97    1.0       0.40  0.919433  12.0
98    1.0       0.45  0.773692  12.0
99    1.0       0.50  0.780344  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.596001  10.0
1     1.0       0.10 -0.292842  10.0
2     1.0       0.15  0.173665  10.0
3     1.0       0.20 -0.719636  10.0
4     1.0       0.25 -0.197307  10.0
..    ...        ...       ...   ...
115  12.0       0.30 -0.732163  10.0
116  12.0       0.35 -0.602587  10.0
117  12.0       0.40 -0.801271  10.0
118  12.0       0.45 -0.760557  10.0
119  12.0       0.50 -0.737582  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [33]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["avg_npmi"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_avg_npmi.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["avg_npmi"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_avg_npmi.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["avg_npmi"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_avg_npmi.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.965534  12.0
1     0.1       0.10  0.971162  12.0
2     0.1       0.15  0.913233  12.0
3     0.1       0.20  0.991144  12.0
4     0.1       0.25  0.937973  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.749416  12.0
96    1.0       0.35  0.772187  12.0
97    1.0       0.40  0.782918  12.0
98    1.0       0.45  0.817629  12.0
99    1.0       0.50  0.832279  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05  0.498072  10.0
1     1.0       0.10 -0.406407  10.0
2     1.0       0.15 -0.229495  10.0
3     1.0       0.20  0.027630  10.0
4     1.0       0.25  0.488408  10.0
..    ...        ...       ...   ...
115  12.0       0.30  0.574959  10.0
116  12.0       0.35  0.893352  10.0
117  12.0       0.40  0.685912  10.0
118  12.0       0.45  0.648706  10.0
119  12.0       0.50  0.589849  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    

In [34]:

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["R0"].values, g["div_all_isolates"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["sigma", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/R0_div_all_isolates.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="sigma", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (R0, Dimmunity) and correlate sigma vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["sigma"].values, g["div_all_isolates"].values)
    return pd.Series({"r": r, "n": n})

res = (
    df.groupby(["R0", "Dimmunity"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/sigma_div_all_isolates.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="Dimmunity", values="r")
print(R_mat.shape)  # (10, 12)


# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["div_all_isolates"].values)
    return pd.Series({"r": r, "n": n})


res = (
    df.groupby(["R0", "sigma"], sort=True, as_index=False)
      .apply(corr_one_group)
      .reset_index(drop=True)
)

# res has 120 rows with columns: x1, x2, r, n
print(res)
np.savetxt("./experimental_data/Dimmunity_div_all_isolates.csv", res, delimiter=",", comments="")
# Optional: matrix form for plotting (rows=x2, cols=x1)
R_mat = res.pivot(index="R0", columns="sigma", values="r")
print(R_mat.shape)  # (10, 12)



    sigma  Dimmunity         r     n
0     0.1       0.05  0.807562  12.0
1     0.1       0.10  0.932924  12.0
2     0.1       0.15  0.935930  12.0
3     0.1       0.20  0.959930  12.0
4     0.1       0.25  0.933428  12.0
..    ...        ...       ...   ...
95    1.0       0.30  0.722281  12.0
96    1.0       0.35  0.710794  12.0
97    1.0       0.40  0.698973  12.0
98    1.0       0.45  0.727353  12.0
99    1.0       0.50  0.748690  12.0

[100 rows x 4 columns]
(10, 10)
       R0  Dimmunity         r     n
0     1.0       0.05 -0.673178  10.0
1     1.0       0.10  0.702256  10.0
2     1.0       0.15  0.277637  10.0
3     1.0       0.20  0.237265  10.0
4     1.0       0.25 -0.337872  10.0
..    ...        ...       ...   ...
115  12.0       0.30  0.851226  10.0
116  12.0       0.35  0.941221  10.0
117  12.0       0.40  0.829132  10.0
118  12.0       0.45  0.784665  10.0
119  12.0       0.50  0.763710  10.0

[120 rows x 4 columns]
(12, 10)
       R0  sigma         r     n
0     1.0    